# **I .Introduction** #

---

<font size = '4'> This notebook utilizes LSTM and generative AI to forecast the quantity of products for the next quarter based on time series data. Since the data exhibits temporal patterns with annual regularity (specific to each store and corresponding product), LSTM and generative models can be effectively used for prediction.
    
<font size = '4'> Reference Previous NoteBook:https://www.kaggle.com/code/p7476762/pivot-lstm-for-predict-future-sales

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
import matplotlib.pyplot as plt
plt.style.use('ggplot')
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# **II .Load The Data and Observe Data Features** #

---



In [ ]:
train_df = pd.read_csv("/kaggle/input/competitive-data-science-predict-future-sales/sales_train.csv")
test_df = pd.read_csv("/kaggle/input/competitive-data-science-predict-future-sales/test.csv")
items_df = pd.read_csv("/kaggle/input/competitive-data-science-predict-future-sales/items.csv")
items_categories_df = pd.read_csv("/kaggle/input/competitive-data-science-predict-future-sales/item_categories.csv")
shops_df = pd.read_csv("/kaggle/input/competitive-data-science-predict-future-sales/shops.csv")

In [ ]:
print(f'The # of nan :{train_df.isna().sum().sum()} in train data')
print(f'The # of nan :{test_df.isna().sum().sum()} in test data')
print(f'The # of nan :{items_df.isna().sum().sum()} in items data')
print(f'The # of nan :{items_categories_df.isna().sum().sum()} in items_categories data')
print(f'The # of nan :{shops_df.isna().sum().sum()} in shops data')

In [ ]:
train_df.head()

In [ ]:
test_df.head()

# **III. Data Extraction (Pivot-Table)** #

---
                            

In [ ]:
train_data = train_df.copy()
test_Data = test_df.copy()
test_Data['date_block_num'] = 34
test_Data['item_cnt_day'] = 0

In [ ]:
train_Data = train_data.pivot_table(index = ['shop_id','item_id'],values = ['item_cnt_day'],columns = ['date_block_num'],fill_value = 0,aggfunc='sum') 
test_Data = test_Data.pivot_table(index = ['shop_id','item_id'],values = ['ID'],columns = ['date_block_num'],fill_value = 0,aggfunc='sum')
test_Data = test_Data.sort_values(by=('ID', 34), ascending=True)

In [ ]:
Combine_train_test = pd.merge(test_Data,train_Data, how = 'left', on = ['shop_id','item_id']).fillna(0)
Combine_train_test = Combine_train_test.drop(columns = (          'ID', 34))

In [ ]:
Combine_train_test.head(15)

# **IV. Data - Targer Engineer** #

In [ ]:
Combine_train_test.shape

In [ ]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler(feature_range=(0, 1))
Combine_train_test = scaler.fit_transform(Combine_train_test)

In [ ]:
#Train Data
train_data = np.array(Combine_train_test[:,:-1]).reshape(np.array(Combine_train_test[:,:-1]).shape[0],np.array(Combine_train_test[:,:-1]).shape[1], 1)
#The target 
train_target = Combine_train_test[:,-1:]

test_data = np.array(Combine_train_test[:,1:]).reshape(np.array(Combine_train_test[:,1:]).shape[0],np.array(Combine_train_test[:,1:]).shape[1], 1)

In [ ]:
#date_block_num = 33
train_data[0].reshape(1,33)

In [ ]:
#date_block_num = 1~33
test_data[0:1].reshape(1,33)

# **IV. LSTM Training** #

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Dropout, Conv1D, MaxPooling1D, TimeDistributed, Flatten

model = Sequential()
model.add(LSTM(50, return_sequences = True, input_shape = (train_data.shape[1], 1)))
model.add(LSTM(50, return_sequences = False ))
model.add(Dense(1))
model.compile(loss = 'mse', optimizer = 'adam', metrics = ['mse'])

In [ ]:
history_1 = model.fit(train_data, train_target, epochs = 10, batch_size = 128)
df_his1 = pd.DataFrame(history_1.history)
plt.figure(figsize = (10 , 10))
sns.lineplot(x= df_his1.index + 1, y = df_his1['loss'], color='r', label="LSTM loss")

In [ ]:
from prophet import Prophet



# **V. LSTM Training** #

In [ ]:
submission = pd.read_csv('/kaggle/input/competitive-data-science-predict-future-sales/sample_submission.csv')
predicted_sales = model.predict(test_data)
original_shape = Combine_train_test.shape
expanded_predicted_sales = np.zeros((original_shape[0], original_shape[1])) 
expanded_predicted_sales[:, -1] = predicted_sales.flatten()
item_cnt_month = scaler.inverse_transform(expanded_predicted_sales)
submission['item_cnt_month'] = item_cnt_month[:, -1].clip(0,30).round(0)
submission.to_csv('submission.csv', index=False)

In [ ]:
submission